import

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import cmath
import time
import class_informs as inform

데이터 조정치

In [2]:
#timeseries data
all_data=pd.read_csv('Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()



dic_data = {k: v for k, v in all_data.groupby('date')}

using_dic = dic_data[pd.to_datetime(date_list[0])]



asset_list = []
for i in using_dic["sedol"]:
    asset_list.append(i)


dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }
dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

for i in using_dic.index:
    dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
    dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])


#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])



sedol_var_list =[]

for i in risk_sedol:
    sedol_var_list.append("d"+str(i))

for i in risk_sedol:
    sedol_var_list.append("q"+str(i))



sedol_var_list.append("assum")
alpha = []


for i in risk_sedol:

    alpha.append(-1*dic_sedol_as[i])

qmat=[]





for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(10000*j)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(len(risk_mat)):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)

for i in range(1):
    qmat_1=[]
    qmat_1.append(sedol_var_list)
    new_risk_mat=[]
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    for j in risk_mat[i]:
        new_risk_mat.append(0)
    new_risk_mat.append(0)
    qmat_1.append(new_risk_mat)
    qmat.append(qmat_1)






q_con1 = []
q_con2 = []
q_val = []


for i in range(len((risk_mat[0]))):
    for j in range(len(risk_mat[0])):
        if j >= i:
            q_con1.append(i)
            q_con2.append(j)
            if i == j:
                ex_list = list(risk_mat[i])
                q_val.append(0.5*ex_list[j]*10000)
            else:
                ex_list = list(risk_mat[i])
                q_val.append(ex_list[j]*10000)



Q_con = []
Q_con.append(q_con1)
Q_con.append(q_con2)
Q_con.append(q_val)



# dic_result = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000)

함수 정의

In [3]:
def cal_obj(sol, risk_mat, alpha) :
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, alpha)
    obj = s_r+s_a
    return obj*10000


def select_candi(sol) :
    diff_add = np.array([[0]*len(alpha)]*len(alpha), float)
    
    pre = cal_obj(sol, risk_mat, alpha)
    
    sol = np.array(sol)
#     a = np.array(sol[sol>=0.001])
#     aa = np.sort(a)[::-1]
    not_zero_sol = np.argsort(sol)[::-1][:len(sol[sol>=0.001])]
    zero_sol = np.argsort(sol)[:len(sol[sol<0.001])]
    
    for i in not_zero_sol :
        w = sol[i]
        sol[i] = 0
        for j in zero_sol :
            w1 = sol[j]
            sol[j] = w
#             sol[j] = 0.0001
            diff_add[i][j] = cal_obj(sol, risk_mat, alpha) - pre
            sol[j] = w1
        sol[i] = w
    
    row, col = np.where(diff_add == np.amin(diff_add[diff_add!=0.]))

    sol[row] = 0
    sol[col] = 0.1
    
    candi = np.argsort(sol)[::-1][:len(sol[sol>=0.001])]
    
    return candi


def update(sol) :
    start_time = time.time()
    ex = inform.informs(10000)
    ex.set_omega(risk_mat)
    
    sol_list = []
    for k in risk_sedol :
        sol_list.append(sol[k])
    result_list = []
    
    while(True) :
        if time.time() - start_time > 50:
            break
        con = select_candi(sol_list)
        ex.set_con(cons=con)
        sol_list2 = []
        try:
            sol2 = ex.solve(0.01)
            if sol2[0] != 0:
                for k in risk_sedol :
                    sol_list2.append(sol2[0][k])
                sol_list = sol_list2
                result_list.append(sol2[2])
            else:
                print("infeasible solution")
        except:
            print("no solution")
            
    return result_list, sol_list

초기 cplex 실행

In [4]:
sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000, time_init=30.0)[0]

Tried aggregator 2 times.
MIQCP Presolve eliminated 1486 rows and 2 columns.
MIQCP Presolve modified 1476 coefficients.
Aggregator did 58 substitutions.
Reduced MIQCP has 1939 rows, 1913 columns, and 196670 nonzeros.
Reduced MIQCP has 492 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQCP has 2 quadratic constraints.
Presolve time = 0.23 sec. (315.46 ticks)
Probing time = 0.05 sec. (24.72 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.06 sec. (51.89 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0     0       -0.0000     0                     -0.0001     1158         
      0     0       -0.0000     3                     Cone: 2     1474         
      0     0       -0.0000     0                     Cone: 4     1600         
      0     0       -0.0000    

선택 자산 update

In [5]:
result = update(sol)

print("objective value 변화")
print(result[0])
# print("portfolio's weight")
# print(result[1])

C:\ProgramData\Anaconda3\envs\py34\lib\site-packages\numpy\core\numeric.py:301: FutureWarning: in the future, full((492, 492), 10000) will return an array of dtype('int32')
  format(shape, fill_value, array(fill_value).dtype), FutureWarning)


0
1
2
3
4
5
no solution
6
no solution
7
no solution
8
no solution
9
no solution
objective value 변화
[11.210605012822587, 12.498154236391187, 13.91436959274973, 17.405042803196533, 26.102689537296417]
